mxnet_1_lineRegression的label（y）是单值。本节课是多分类得分

1.下载数据：

In [ ]:
from mxnet import gluon
from mxnet import ndarray as nd

def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')

mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test  = gluon.data.vision.FashionMNIST(train=False, transform=transform)
print (type(mnist_train))
#print (mnist_train.shape)

In [ ]:
data, label = mnist_train[0]
print ('example shape: ', data.shape, 'label:', label)

#help(gluon.data.vision.FashionMNIST)
print(data[3]) #28*1
print(data[3].reshape((28,)))#28

2.准备数据

In [ ]:
import matplotlib.pyplot as plt

def show_images(images):
    n = images.shape[0]
    _, figs = plt.subplots(1, n, figsize=(15, 15))
    for i in range(n):
        figs[i].imshow(images[i].reshape((28,28)).asnumpy())
        figs[i].axes.get_xaxis().set_visible(False)
        figs[i].axes.get_yaxis().set_visible(False)
    plt.show()

def get_text_labels(label):
    text_labels = [
        't-shirt', 'trouser', 'pullover', 'dress,', 'coat',
        'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'
    ]
    return [text_labels[int(i)] for i in label]

data, label = mnist_train[0:9]
show_images(data)

batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data  = gluon.data.DataLoader(mnist_test,  batch_size, shuffle=False)

3.准备参数

In [ ]:
num_inputs  = 28*28
num_output = 10

w = nd.random_normal(shape=(num_inputs,num_output))
b = nd.random_normal(shape=num_output)
params = [w, b]

for param in params:
    param.attach_grad()

4.定义网络

In [ ]:
def softmax(x):
    exp = nd.exp(x)
    partition = exp.sum(axis=1, keepdims=True)
    return exp/partition

In [ ]:
def net(X):
    #reshape是输入的X是3D的，即batch_size*28*28的数据,reshape为batch_size*284
    return softmax(nd.dot(X.reshape((-1,num_inputs)),w) + b)

5.定义Loss函数

In [ ]:
#实际值：yvec=[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
#预测值：yhat=[.....]
#交叉熵 yvec[0]*log(yhat[0])+...+yvec[n]*log(yhat[n])
#相乘后为：log(yhat[y])
def cross_entropy(yhat, y):
    return -nd.pick(nd.log(yhat),y)

def accuracy(output, label):
    #output 256个10维向量
    #label 256向量
    return nd.mean(output.argmax(axis=1)==label).asscalar()

def evaluate_accuracy(data_iterator, net):
    acc = 0.
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)

evaluate_accuracy(test_data, net)

6.训练

In [ ]:
import sys
sys.path.append('..')
from utils import SGD #是同目录下的utils.py文件
import mxnet.autograd as ag

lr = .01
epochs = 10

for epoch in range(epochs):
    train_loss = 0.
    train_acc  = 0.
    for data, label in train_data:
        with ag.record():
            output = net(data)
            loss   = cross_entropy(output, label)
        loss.backward()
        SGD(params, lr/batch_size)
        
        train_loss += nd.mean(loss).asscalar()
        train_acc  += accuracy(output, label)
    
    test_acc = evaluate_accuracy(test_data, net)
    print ("Epoch %d. Loss %f. Acc %f. Test Acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))